In [1]:
import sys
sys.path.insert(0, "../")


In [2]:
from pathlib import Path 
import requests

#Download helper function from PyTorch repo
if Path('utils.py').is_file():
    print("lxmert_utils already exists")
else:
    request = requests.get("https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/lxmert/utils.py")
    with open("utils.py", "wb") as f:
        f.write(request.content)

if Path('modeling_frcnn.py').is_file():
    print("modeling_frcnn already exists")
else:
    request = requests.get("https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/lxmert/modeling_frcnn.py")
    with open("modeling_frcnn.py", "wb") as f:
        f.write(request.content)

if Path('processing_image.py').is_file():
    print("processing_image already exists")
else:
    request = requests.get("https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/lxmert/processing_image.py")
    with open("processing_image.py", "wb") as f:
        f.write(request.content)
from modeling_frcnn import GeneralizedRCNN

lxmert_utils already exists
modeling_frcnn already exists
processing_image already exists


c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from utils import Config
import transformers
from processing_image import Preprocess
from modeling_frcnn import GeneralizedRCNN
from transformers import LxmertTokenizer, LxmertForPreTraining
import torch
import numpy as np

Load necessary functions for images preprocessing to generate image features and bounding boxes

In [4]:
config_frcnn = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")
frcnnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=config_frcnn)
image_preprocess = Preprocess(config_frcnn)
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'

loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at C:\Users\DELL/.cache\torch\transformers\57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [5]:
tokenizer = LxmertTokenizer.from_pretrained('unc-nlp/lxmert-base-uncased')
model = LxmertForPreTraining.from_pretrained('unc-nlp/lxmert-base-uncased')

In [6]:
def get_matching_score(img, caption):
    images, sizes, scales_yx = image_preprocess(img)
    output_dict = frcnnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=config_frcnn.max_detections,
        return_tensors="pt",
    )
    normalized_boxes = output_dict.get("normalized_boxes")
    features = output_dict.get("roi_features")
    tokenized = tokenizer(caption, padding=True)
    model_input={}
    model_input['input_ids'] = torch.tensor(tokenized['input_ids'], dtype=torch.long)[None,:].to(device)
    model_input['attention_mask'] = torch.tensor(tokenized['attention_mask'])[None,:].to(device)
    model_input['token_type_ids'] = torch.tensor(tokenized['token_type_ids'], dtype=torch.long)[None,:].to(device)
    model_input['visual_feats'] = features
    model_input['visual_pos'] = normalized_boxes
    with torch.inference_mode():
        output = model(**model_input, output_attentions=True)
    del model_input, normalized_boxes, features, output_dict, images, sizes, scales_yx
    itm_scores = output.cross_relationship_score[0] 
    itm_scores_prob = torch.nn.functional.softmax(itm_scores, dim=0)
    return itm_scores_prob[1].item()

In [7]:
def success_score(img_0, img_1, cap_0, cap_1):
    score_img_0_cap_0 = get_matching_score(img_0, cap_0)
    score_img_0_cap_1 = get_matching_score(img_0, cap_1)
    score_img_1_cap_0 = get_matching_score(img_1, cap_0)
    score_img_1_cap_1 = get_matching_score(img_1, cap_1)
    file = open("../output_files/lxmert_wino_match_score.txt", "a")
    file.write(f"img-cap(0,0): {score_img_0_cap_0:.5}, img-cap(0,1): {score_img_0_cap_1:.5}, img-cap(1,0): {score_img_1_cap_0:.5}, img-cap(1,1): {score_img_1_cap_1:.5}\n")
    group_score = False 
    if (score_img_0_cap_0>score_img_0_cap_1) and (score_img_1_cap_1>score_img_1_cap_0):
        group_score=True
    return group_score

In [8]:
from source.wino_loader import extract_captions_winoground
winoground = extract_captions_winoground(dir = "../dataset/examples.jsonl")
hit_score = 0
for i in range(len(winoground['caption_0'])):
    img_0 = "../dataset/images/"+ winoground["image_0"][i]+".png"
    img_1 = "../dataset/images/"+ winoground["image_1"][i]+".png"
    caption_0 = winoground["caption_0"][i]
    caption_1= winoground["caption_1"][i]
    group_score =success_score(img_0, img_1, caption_0, caption_1)
    if group_score:
        hit_score +=1
print(f"Hit Rate: {hit_score/len(winoground['caption_0'])}")

c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Hit Rate: 0.2075


### THIS PORTION CONTAINS THE CODE TO COMPARE THE PERFORAMNCE OF CLIP MODEL WITH WINOGROUND AND MODIFIED WINOGROUND DATASET ###
*MODIFIED CAPTIONS = 50*

In [1]:
import sys
sys.path.insert(0, "../")


from pathlib import Path 
import requests

#Download helper function from PyTorch repo
if Path('utils.py').is_file():
    print("lxmert_utils already exists")
else:
    request = requests.get("https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/lxmert/utils.py")
    with open("utils.py", "wb") as f:
        f.write(request.content)

if Path('modeling_frcnn.py').is_file():
    print("modeling_frcnn already exists")
else:
    request = requests.get("https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/lxmert/modeling_frcnn.py")
    with open("modeling_frcnn.py", "wb") as f:
        f.write(request.content)

if Path('processing_image.py').is_file():
    print("processing_image already exists")
else:
    request = requests.get("https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/lxmert/processing_image.py")
    with open("processing_image.py", "wb") as f:
        f.write(request.content)
from modeling_frcnn import GeneralizedRCNN


from utils import Config
import transformers
from processing_image import Preprocess
from modeling_frcnn import GeneralizedRCNN
from transformers import LxmertTokenizer, LxmertForPreTraining
import torch
import numpy as np


config_frcnn = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")
frcnnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=config_frcnn)
image_preprocess = Preprocess(config_frcnn)
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'

tokenizer = LxmertTokenizer.from_pretrained('unc-nlp/lxmert-base-uncased')
model = LxmertForPreTraining.from_pretrained('unc-nlp/lxmert-base-uncased')

def get_matching_score(img, caption):
    images, sizes, scales_yx = image_preprocess(img)
    output_dict = frcnnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=config_frcnn.max_detections,
        return_tensors="pt",
    )
    normalized_boxes = output_dict.get("normalized_boxes")
    features = output_dict.get("roi_features")
    tokenized = tokenizer(caption, padding=True)
    model_input={}
    model_input['input_ids'] = torch.tensor(tokenized['input_ids'], dtype=torch.long)[None,:].to(device)
    model_input['attention_mask'] = torch.tensor(tokenized['attention_mask'])[None,:].to(device)
    model_input['token_type_ids'] = torch.tensor(tokenized['token_type_ids'], dtype=torch.long)[None,:].to(device)
    model_input['visual_feats'] = features
    model_input['visual_pos'] = normalized_boxes
    with torch.inference_mode():
        output = model(**model_input, output_attentions=True)
    del model_input, normalized_boxes, features, output_dict, images, sizes, scales_yx
    itm_scores = output.cross_relationship_score[0] 
    itm_scores_prob = torch.nn.functional.softmax(itm_scores, dim=0)
    return itm_scores_prob[1].item()

def success_score(img_0, img_1, cap_0, cap_1):
    score_img_0_cap_0 = get_matching_score(img_0, cap_0)
    score_img_0_cap_1 = get_matching_score(img_0, cap_1)
    score_img_1_cap_0 = get_matching_score(img_1, cap_0)
    score_img_1_cap_1 = get_matching_score(img_1, cap_1)
    file = open("../output_files/lxmert_wino_match_score.txt", "a")
    file.write(f"img-cap(0,0): {score_img_0_cap_0:.5}, img-cap(0,1): {score_img_0_cap_1:.5}, img-cap(1,0): {score_img_1_cap_0:.5}, img-cap(1,1): {score_img_1_cap_1:.5}\n")
    group_score = False 
    if (score_img_0_cap_0>score_img_0_cap_1) and (score_img_1_cap_1>score_img_1_cap_0):
        group_score=True
    return group_score

from source.wino_loader import extract_edited_captions_winoground
n_samples=50
modified_winoground = extract_edited_captions_winoground(dir="../dataset/edited_captions.xls", edited_captions=n_samples)
wino_hit_score = 0
modified_hit_score =0

##----- compare wino and modified outcome-------
wino_0_modified_0 = 0
wino_0_modified_1 = 0
wino_1_modified_0 = 0
wino_1_modified_1 = 0
for i in range(len(modified_winoground['true_caption_0'])):
    img_0 = "../dataset/images/"+ modified_winoground["image_0"][i]+".png"
    img_1 = "../dataset/images/"+ modified_winoground["image_1"][i]+".png"
    true_caption_0 = modified_winoground["true_caption_0"][i]
    true_caption_1= modified_winoground["true_caption_1"][i]
    modified_caption_0 = modified_winoground["modified_caption_0"][i]
    modified_caption_1= modified_winoground["modified_caption_1"][i]
    wino_group_score =success_score(img_0, img_1, true_caption_0, true_caption_1)
    modified_group_score =success_score(img_0, img_1, modified_caption_0, modified_caption_1)
    if wino_group_score:
        wino_hit_score +=1
    if modified_group_score:
        modified_hit_score +=1

    
    if wino_group_score==False and modified_group_score==False:
        wino_0_modified_0 +=1
    elif wino_group_score==False and modified_group_score==True:
        wino_0_modified_1 +=1
    elif wino_group_score==True and modified_group_score==False:
        wino_1_modified_0 +=1
    elif wino_group_score==True and modified_group_score==True:
        wino_1_modified_1 +=1


print(f"Hit Ratio for Winoground: {wino_hit_score/n_samples}")
print(f"Hit Ratio for Modified Winoground: {modified_hit_score/n_samples}")
print(f"Winoground Hit Score: {wino_hit_score}")
print(f"Modified Winoground Hit Score: {modified_hit_score}")
print(f"No. of Samples where BOTH the WINOGROUND captions and MODIFIED pairs are identified INCORRECTLY: {wino_0_modified_0}")
print(f"No. of Samples where the WINOGROUND captions are indentified INCORRECTLY but MODIFIED pairs are identified CORRECTLY: {wino_0_modified_1}")
print(f"No. of Samples where the WINOGROUND captions are indentified CORRECTLY but MODIFIED pairs are identified INCORRECTLY: {wino_1_modified_0}")
print(f"No. of Samples where both the WINOGROUND captions and MODIFIED pairs are identified CORRECTLY: {wino_1_modified_1}")

lxmert_utils already exists
modeling_frcnn already exists
processing_image already exists


c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at C:\Users\DELL/.cache\torch\transformers\57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Hit Ratio for Winoground: 0.16
Hit Ratio for Modified Winoground: 0.32
Winoground Hit Score: 8
Modified Winoground Hit Score: 16
No. of Samples where BOTH the WINOGROUND captions and MODIFIED pairs are identified INCORRECTLY: 30
No. of Samples where the WINOGROUND captions are indentified INCORRECTLY but MODIFIED pairs are identified CORRECTLY: 12
No. of Samples where the WINOGROUND captions are indentified CORRECTLY but MODIFIED pairs are identified INCORRECTLY: 4
No. of Samples where both the WINOGROUND captions and MODIFIED pairs are identified CORRECTLY: 4


### THIS PORTION CONTAINS THE CODE TO COMPARE THE PERFORAMNCE OF CLIP MODEL WITH WINOGROUND AND MODIFIED WINOGROUND DATASET ###
*MODIFIED CAPTIONS = 100*

In [1]:
import sys
sys.path.insert(0, "../")


from pathlib import Path 
import requests

#Download helper function from PyTorch repo
if Path('utils.py').is_file():
    print("lxmert_utils already exists")
else:
    request = requests.get("https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/lxmert/utils.py")
    with open("utils.py", "wb") as f:
        f.write(request.content)

if Path('modeling_frcnn.py').is_file():
    print("modeling_frcnn already exists")
else:
    request = requests.get("https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/lxmert/modeling_frcnn.py")
    with open("modeling_frcnn.py", "wb") as f:
        f.write(request.content)

if Path('processing_image.py').is_file():
    print("processing_image already exists")
else:
    request = requests.get("https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/lxmert/processing_image.py")
    with open("processing_image.py", "wb") as f:
        f.write(request.content)
from modeling_frcnn import GeneralizedRCNN


from utils import Config
import transformers
from processing_image import Preprocess
from modeling_frcnn import GeneralizedRCNN
from transformers import LxmertTokenizer, LxmertForPreTraining
import torch
import numpy as np


config_frcnn = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")
frcnnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=config_frcnn)
image_preprocess = Preprocess(config_frcnn)
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'

tokenizer = LxmertTokenizer.from_pretrained('unc-nlp/lxmert-base-uncased')
model = LxmertForPreTraining.from_pretrained('unc-nlp/lxmert-base-uncased')

def get_matching_score(img, caption):
    images, sizes, scales_yx = image_preprocess(img)
    output_dict = frcnnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=config_frcnn.max_detections,
        return_tensors="pt",
    )
    normalized_boxes = output_dict.get("normalized_boxes")
    features = output_dict.get("roi_features")
    tokenized = tokenizer(caption, padding=True)
    model_input={}
    model_input['input_ids'] = torch.tensor(tokenized['input_ids'], dtype=torch.long)[None,:].to(device)
    model_input['attention_mask'] = torch.tensor(tokenized['attention_mask'])[None,:].to(device)
    model_input['token_type_ids'] = torch.tensor(tokenized['token_type_ids'], dtype=torch.long)[None,:].to(device)
    model_input['visual_feats'] = features
    model_input['visual_pos'] = normalized_boxes
    with torch.inference_mode():
        output = model(**model_input, output_attentions=True)
    del model_input, normalized_boxes, features, output_dict, images, sizes, scales_yx
    itm_scores = output.cross_relationship_score[0] 
    itm_scores_prob = torch.nn.functional.softmax(itm_scores, dim=0)
    return itm_scores_prob[1].item()

def success_score(img_0, img_1, cap_0, cap_1):
    score_img_0_cap_0 = get_matching_score(img_0, cap_0)
    score_img_0_cap_1 = get_matching_score(img_0, cap_1)
    score_img_1_cap_0 = get_matching_score(img_1, cap_0)
    score_img_1_cap_1 = get_matching_score(img_1, cap_1)
    file = open("../output_files/lxmert_wino_match_score.txt", "a")
    file.write(f"img-cap(0,0): {score_img_0_cap_0:.5}, img-cap(0,1): {score_img_0_cap_1:.5}, img-cap(1,0): {score_img_1_cap_0:.5}, img-cap(1,1): {score_img_1_cap_1:.5}\n")
    group_score = False 
    if (score_img_0_cap_0>score_img_0_cap_1) and (score_img_1_cap_1>score_img_1_cap_0):
        group_score=True
    return group_score

from source.wino_loader import extract_edited_captions_winoground
n_samples=100
modified_winoground = extract_edited_captions_winoground(dir="../dataset/edited_captions.xls", edited_captions=n_samples)
wino_hit_score = 0
modified_hit_score =0

##----- compare wino and modified outcome-------
wino_0_modified_0 = 0
wino_0_modified_1 = 0
wino_1_modified_0 = 0
wino_1_modified_1 = 0
for i in range(len(modified_winoground['true_caption_0'])):
    img_0 = "../dataset/images/"+ modified_winoground["image_0"][i]+".png"
    img_1 = "../dataset/images/"+ modified_winoground["image_1"][i]+".png"
    true_caption_0 = modified_winoground["true_caption_0"][i]
    true_caption_1= modified_winoground["true_caption_1"][i]
    modified_caption_0 = modified_winoground["modified_caption_0"][i]
    modified_caption_1= modified_winoground["modified_caption_1"][i]
    wino_group_score =success_score(img_0, img_1, true_caption_0, true_caption_1)
    modified_group_score =success_score(img_0, img_1, modified_caption_0, modified_caption_1)
    if wino_group_score:
        wino_hit_score +=1
    if modified_group_score:
        modified_hit_score +=1

    
    if wino_group_score==False and modified_group_score==False:
        wino_0_modified_0 +=1
    elif wino_group_score==False and modified_group_score==True:
        wino_0_modified_1 +=1
    elif wino_group_score==True and modified_group_score==False:
        wino_1_modified_0 +=1
    elif wino_group_score==True and modified_group_score==True:
        wino_1_modified_1 +=1


print(f"Hit Ratio for Winoground: {wino_hit_score/n_samples}")
print(f"Hit Ratio for Modified Winoground: {modified_hit_score/n_samples}")
print(f"Winoground Hit Score: {wino_hit_score}")
print(f"Modified Winoground Hit Score: {modified_hit_score}")
print(f"No. of Samples where BOTH the WINOGROUND captions and MODIFIED pairs are identified INCORRECTLY: {wino_0_modified_0}")
print(f"No. of Samples where the WINOGROUND captions are indentified INCORRECTLY but MODIFIED pairs are identified CORRECTLY: {wino_0_modified_1}")
print(f"No. of Samples where the WINOGROUND captions are indentified CORRECTLY but MODIFIED pairs are identified INCORRECTLY: {wino_1_modified_0}")
print(f"No. of Samples where both the WINOGROUND captions and MODIFIED pairs are identified CORRECTLY: {wino_1_modified_1}")

lxmert_utils already exists
modeling_frcnn already exists
processing_image already exists


c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at C:\Users\DELL/.cache\torch\transformers\57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Hit Ratio for Winoground: 0.13
Hit Ratio for Modified Winoground: 0.3
Winoground Hit Score: 13
Modified Winoground Hit Score: 30
No. of Samples where BOTH the WINOGROUND captions and MODIFIED pairs are identified INCORRECTLY: 64
No. of Samples where the WINOGROUND captions are indentified INCORRECTLY but MODIFIED pairs are identified CORRECTLY: 23
No. of Samples where the WINOGROUND captions are indentified CORRECTLY but MODIFIED pairs are identified INCORRECTLY: 6
No. of Samples where both the WINOGROUND captions and MODIFIED pairs are identified CORRECTLY: 7


In [7]:
images, sizes, scales_yx = image_preprocess('../dataset/images/ex_0_img_0.png')
output_dict = frcnnn(
    images,
    sizes,
    scales_yx=scales_yx,
    padding="max_detections",
    max_detections=config_frcnn.max_detections,
    return_tensors="pt",
)

c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
# Very important that the boxes are normalized
normalized_boxes = output_dict.get("normalized_boxes")
features = output_dict.get("roi_features")

In [9]:
features.shape

torch.Size([1, 36, 2048])

loading file vocab.txt from cache at C:\Users\DELL/.cache\huggingface\hub\models--unc-nlp--lxmert-base-uncased\snapshots\628572c96242d1496147beec1c13a1bb7869605d\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\DELL/.cache\huggingface\hub\models--unc-nlp--lxmert-base-uncased\snapshots\628572c96242d1496147beec1c13a1bb7869605d\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\DELL/.cache\huggingface\hub\models--unc-nlp--lxmert-base-uncased\snapshots\628572c96242d1496147beec1c13a1bb7869605d\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\DELL/.cache\huggingface\hub\models--unc-nlp--lxmert-base-uncased\snapshots\628572c96242d1496147beec1c13a1bb7869605d\config.json
Model config LxmertConfig {
  "_name_or_path": "unc-nlp/lxmert-base-uncased",
  "architectures": [
    "LxmertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidd

In [50]:
device = 'cpu'
model_input={}
caption ='an old person kisses a young person'
tokenized = tokenizer(caption, padding=True)
model_input['input_ids'] = torch.tensor(tokenized['input_ids'], dtype=torch.long)[None,:].to(device)
model_input['attention_mask'] = torch.tensor(tokenized['attention_mask'])[None,:].to(device)
model_input['token_type_ids'] = torch.tensor(tokenized['token_type_ids'], dtype=torch.long)[None,:].to(device)
model_input['visual_feats'] = features
model_input['visual_pos'] = normalized_boxes


In [51]:
output = model(**model_input, output_attentions=True)


In [52]:
output.keys()

odict_keys(['prediction_logits', 'cross_relationship_score', 'question_answering_score', 'language_attentions', 'vision_attentions', 'cross_encoder_attentions'])

In [42]:
output.pooled_output.shape
torch.functional.Sigmoid(output.pooled_output)

AttributeError: module 'torch.functional' has no attribute 'Sigmoid'

In [53]:
itm_scores = output.cross_relationship_score[0] # Batch size 1 so [0]
itm_scores_prob = torch.nn.functional.softmax(itm_scores, dim=0)
itm_scores_prob[1].item()

0.14158210158348083

In [56]:
output.cross_relationship_score[0]

tensor([ 0.8916, -0.9106], grad_fn=<SelectBackward0>)

In [57]:
itm_scores_prob

tensor([0.8584, 0.1416], grad_fn=<SoftmaxBackward0>)

https://github.com/huggingface/transformers/issues/7266#issuecomment-748211135
This comment saves my life